In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('drive/MyDrive/Аналітик даних в ІТ/Data/supermarket_sales.csv')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
# Перегляд датафрейму
df.head(3)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4


In [35]:
# Створення зведеної таблиці 1
pivot_table = pd.pivot_table(df,
                             index = ['Customer type','Gender'],
                             columns = 'City',
                             values = 'Invoice ID',
                             aggfunc = 'count')
pivot_table

City                  Mandalay  Naypyitaw  Yangon
Customer type Gender                             
Member        Female        85         96      80
              Male          80         73      87
Normal        Female        77         82      81
              Male          90         77      92

In [36]:
# Обчислення всього транзакцій в розрізі міст
pivot_table.sum(axis=0)

,0
City,
Mandalay,332
Naypyitaw,328
Yangon,340


In [37]:
# Обчислення і додавання нового рядка "Всього" до першої зведеної таблиці

pivot_table.loc['Всього'] = pivot_table.sum(numeric_only=True)
pivot_table

City,Mandalay,Naypyitaw,Yangon
"(Member, Female)",85,96,80
"(Member, Male)",80,73,87
"(Normal, Female)",77,82,81
"(Normal, Male)",90,77,92
Всього,332,328,340


In [13]:
# Обчислення долі кількості транзакцій для Customer type + Gender в розрізі міст і
# створення нової зведеної таблиці 2

perc_pivot = pivot_table.div(pivot_table.sum(axis=0),axis=1)*100
perc_pivot

City                   Mandalay  Naypyitaw     Yangon
Customer type Gender                                 
Member        Female  25.602410  29.268293  23.529412
              Male    24.096386  22.256098  25.588235
Normal        Female  23.192771  25.000000  23.823529
              Male    27.108434  23.475610  27.058824

In [38]:
# Обчислення і додавання нового рядка "Всього" до другої зведеної таблиці

perc_pivot.loc['Всього'] = perc_pivot.sum(numeric_only=True)
perc_pivot

City,Mandalay,Naypyitaw,Yangon
"(Member, Female)",25.602410,29.268293,23.529412
"(Member, Male)",24.096386,22.256098,25.588235
"(Normal, Female)",23.192771,25.000000,23.823529
"(Normal, Male)",27.108434,23.475610,27.058824
Всього,100.000000,100.000000,100.000000


In [39]:
# Переформатування значень долі з float в int та додавання "%"

percentages = perc_pivot.round(0).astype(int).astype(str) + '%'
percentages

City,Mandalay,Naypyitaw,Yangon
"(Member, Female)",26%,29%,24%
"(Member, Male)",24%,22%,26%
"(Normal, Female)",23%,25%,24%
"(Normal, Male)",27%,23%,27%
Всього,100%,100%,100%


In [40]:
# Об'єднання двох таблиць 1 та 2

concatenated_df = pd.concat([pivot_table, percentages],
                            keys = ['Кількість','Доля %'],
                            axis=1)
concatenated_df

Кількість                    Доля %                 
City              Mandalay Naypyitaw Yangon Mandalay Naypyitaw Yangon
(Member, Female)        85        96     80      26%       29%    24%
(Member, Male)          80        73     87      24%       22%    26%
(Normal, Female)        77        82     81      23%       25%    24%
(Normal, Male)          90        77     92      27%       23%    27%
Всього                 332       328    340     100%      100%   100%

In [41]:
# Перетворення індексу, який складається зі списку кортежів, у багаторівневий індекс з іменами рівнів:
# ФІНАЛЬНАЯ ТАБЛИЦЯ

new_index = []
for idx in concatenated_df.index:
    if isinstance(idx, tuple):
        # звичайні рядки типу (Member, Female)
        new_index.append(idx)
    else:
        # рядок 'Всього' → робимо кортеж ('Всього', '')
        new_index.append((idx, ''))

concatenated_df.index = pd.MultiIndex.from_tuples(
    new_index,
    names=['Customer type', 'Gender']
)
concatenated_df

Кількість                    Доля %                 
City                  Mandalay Naypyitaw Yangon Mandalay Naypyitaw Yangon
Customer type Gender                                                     
Member        Female        85        96     80      26%       29%    24%
              Male          80        73     87      24%       22%    26%
Normal        Female        77        82     81      23%       25%    24%
              Male          90        77     92      27%       23%    27%
Всього                     332       328    340     100%      100%   100%